Workflow used for:
  1. Load metadata about GRSs used
  1. Read phenotypic information and consolidate it with raw PGS scores from **apply_grs.ipynb**
  1. Standardize raw scores and calculate Odds Ratios between case/controls
  1. Generate visualization plots

In [ ]:
# Import all necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import seaborn as sns
import numpy as np
import os, glob
from sklearn import preprocessing
import statsmodels.api as sm
import json
from xlsxwriter.utility import xl_rowcol_to_cell
import glob
import gzip
import zepid
from zepid.graphics import EffectMeasurePlot
import matplotlib.lines as mlines
from statsmodels.stats.multitest import fdrcorrection
import statistics
from scipy.stats import iqr


**Step 1)** Load metadata about GRSs used

In [ ]:
os.chdir("/labs/tassimes/rodrigoguarischi/projects/sea/apply_grs/pgs_reference_weights/")

# Load all key/values present on the header of PGS files into a dictionary
pgs_metadata = {}
for current_score_file in glob.glob( "*.txt.gz" ):
    score_name = current_score_file.replace(".txt.gz","")
    pgs_metadata[score_name] = {}
    with gzip.open( current_score_file,'rt') as f:
        for line in f:
            if( ( line.startswith("#") ) & (not line.startswith("##") ) ):
                line = line.strip()
                line = line.replace("#", "")
                key, value = line.split("=")
                pgs_metadata[score_name][key] = value
    if( str( pgs_metadata[score_name]["pgs_id"] ).startswith("PGS") ):
        pgs_metadata[score_name]["pgs_catalog_hyperlink"] = "https://www.pgscatalog.org/score/{0}/".format(pgs_metadata[score_name]["pgs_id"])
    else:
        pgs_metadata[score_name]["pgs_catalog_hyperlink"] = ""

**Step 2)** Read phenotypic information

In [ ]:
os.chdir("/labs/tassimes/rodrigoguarischi/projects/sea/")

# Read phenotypes and recode sex and race attributes
sea_phenotypes = pd.read_table(
    "data_preparation_to_imputation/86679/NHLBI/SEA_Herrington/phs000349v1/p1/phenotype/phs000349.v1.pht002191.v1.p1.c1.SEA_Phase2_Subject_Phenotypes.GRU.txt",
    index_col="seaid",
    comment="#")

# Recode Sex and Race
sea_phenotypes = sea_phenotypes.replace( {
    "sex": { 1:"male", 2:"female" },
    "race": { 1:"white", 2:"black" }
    } )

# Cast BMI values to integers
sea_phenotypes = sea_phenotypes.assign( bmi=pd.to_numeric(sea_phenotypes["bmi"], errors="coerce") )

# Drop unnecessary column
sea_phenotypes = sea_phenotypes.drop("dbGaP SubjID", axis = 1)

# Print counts by race and sex and first lines from dataframe 
print( "Statistics by subgroups:" )
print( sea_phenotypes.groupby(["race","sex"])["sex"].count(), "\n" )

# Print some summary statistics
phenotype = "cr"

for race in [ "white", "black"]:
    print( "For " + race + " (" + str( len( sea_phenotypes[ sea_phenotypes["race"] == race ] ) ) + " subjects)" )

    age_values = sea_phenotypes[ sea_phenotypes["race"] == race ]["age"]
    print( "  Age: mean {0:.1f}+-{1:.1f}; median {2:.1f} ".format( age_values.mean(), statistics.stdev( age_values ), age_values.median() ) )

    # BMI dropning NA values (3 subjects)
    bmi_values = sea_phenotypes[ sea_phenotypes["race"] == race ]["bmi"].dropna()
    print( "  BMI: mean {0:.1f}+-{1:.1f}; median {2:.1f} ".format( bmi_values.mean(), statistics.stdev( bmi_values ), bmi_values.median() ) )

    print( "  # of males: " + str( len( sea_phenotypes[ ( sea_phenotypes["race"] == race ) & ( sea_phenotypes["sex"] == "male" ) ] )  ) )
    print( "  # of subjects with RCA = 0: " + str(len( sea_phenotypes[ ( sea_phenotypes["race"] == race ) & ( sea_phenotypes[phenotype] == 0 ) ] ) ) )
    
    surface_area_values = sea_phenotypes[ ( sea_phenotypes["race"] == race ) & ( sea_phenotypes[phenotype] > 0 ) ][phenotype]
    print( "  Of the subjects with RCA > 0, the surface area involvement was mean = {0:.1f}% (SD = {1:.1f}); median = {2:.1f}% (IQR = {3:.1f})\n".format(
        surface_area_values.mean(),
        statistics.stdev( surface_area_values ),
        surface_area_values.median(),
        iqr(surface_area_values)
        ) )
    

# Print final dataframe
sea_phenotypes

# plt.rcParams['figure.dpi']= 150
# sns.set_style('white')

# sns.violinplot(
#     x = "race",
#     y = "bmi_i",
#     hue = "sex",
#     hue_order=['female','male'],
#     split = True,
#     bw=0.5,
#     scale = "count",
#     data = sea_dataset_full
#     )

Step 3) Load **PCA results** to dataframe

In [ ]:
os.chdir("/labs/tassimes/rodrigoguarischi/projects/sea/pca_analysis/")

pca_results = {}

pca_results["whites"] = pd.read_table( "pca_whites.eigenvec", sep = "\t" )
pca_results["whites"] = pca_results["whites"].set_index("IID")

pca_results["blacks"] = pd.read_table( "pca_blacks.eigenvec", sep = "\t" )
pca_results["blacks"] = pca_results["blacks"].set_index("IID")

pca_results["all"] = pd.concat( [pca_results["whites"], pca_results["blacks"]], axis=0 ).drop("#FID", axis = 1)

sea_dataset_full = pd.concat( [sea_phenotypes, pca_results["all"]], axis = 1 )
sea_dataset_full

Step 4) Consolidate raw **PGS scores**

In [ ]:
# Read files with raw scores of multiple GRSs

li = []
li.append(sea_dataset_full)

os.chdir("/labs/tassimes/rodrigoguarischi/projects/sea/apply_grs/raw_scores_20220601/")

# Loop over info files and save info as a dictionary   
scores_info_dict = {}
for current_info_file in glob.glob( "*.info.txt" ):

    # Parse filename to get info about GRSs.
    # Format should be: <hrc|topmed>_<whites|blacks>_<8-digits_date>_multiGRS_<minr2_used>.info.txt
    min_r2_used = os.path.basename( current_info_file ).split(".")[0].split("_")[-1]
    reference_panel_name = os.path.basename( current_info_file ).split("_")[0]
    race = os.path.basename( current_info_file ).split("_")[1]
    
    f = open( current_info_file )
    data = json.load(f)

    # Load metrics on a dictionary of dictionaries using panel, r2 and PGS name and each race as keys
    for score_result in data:
        key = "_".join( (reference_panel_name, min_r2_used, score_result["name"]) )
        
        # If key doesn't exist yet in the dictionary, create it
        if key not in scores_info_dict:
            scores_info_dict[key] = {}
            
        scores_info_dict[key][race] = score_result

    f.close()

# Loop over scores files, load all into memory and save it to list object 'li'    
for current_score_file in glob.glob( "*.scores.txt" ):
        
    grs_results = pd.read_table( current_score_file, sep = "," )
    grs_results["sample"] = grs_results["sample"].str.split("_", expand = True)[0]
    grs_results = grs_results.set_index("sample")
    min_r2_used = os.path.basename( current_score_file ).split(".")[0].split("_")[-1]
    reference_panel_name = os.path.basename( current_score_file ).split("_")[0]
        
    # Add prefix to column names. Names should match pattern <hrc|topmed>_<threshold>_<scoreid>
    grs_results = grs_results.add_prefix(reference_panel_name + "_" + min_r2_used + "_")

    # Test if this set of scores already exists in li. 
    # If the other race was already loaded, append subjects to the dataframe. Otherwise, append dataframe to li
    new_score_set = True
    for i in range( len(li) ):
        if( set( grs_results.columns ) == set( li[i].columns ) ):
            li[i] = pd.concat( [li[i], grs_results], axis=0 )
            new_score_set = False

    if( new_score_set ):
        li.append( grs_results )

# Consolidate data into a dataframe and print first lines
sea_dataset_full = pd.concat( li, axis=1 )
sea_dataset_full

Step 5) Assign **case/control** cases

In [ ]:
# # Assign case/control classes to samples using top quartile rule splitting by sex

# # Phenotype of interest
# phenotype = "cr"

# # Calculate thresholds for each race and sex
# print( "Thresholds to be used for case/control definition:" )
# thresholds = {}
# for race in ["white", "black"]:
#     for sex in ["male", "female"]:
        
#         # Define Q3 as threshold and save it in thresholds dictionary
#         key = race + "_" + sex
#         thresholds[ key ] = sea_dataset_full[ (( sea_dataset_full["race"]==race ) & ( sea_dataset_full["sex"]==sex ))  ][phenotype].quantile(0.75)
        
#         # Print values and warnings, if needed
#         note = ""
#         if( thresholds[ key ] == 0 ):
#             note = "(WARNING: This group has Q3 equals zero. Considering only non-zeros as CASE group!!)"
#         print( " - {0} = {1:.4f} {2}".format( key, thresholds[key], note ) )

# # Add a new column called Case with all values equals to False
# sea_dataset_full = sea_dataset_full.assign( Case=False )

# # Identify subjects above threshold to assign them to group "Case"
# for i in sea_dataset_full.index:

#     sex = sea_dataset_full.loc[ i, "sex"]
#     race = sea_dataset_full.loc[ i, "race"]
#     key = race + "_" + sex
    
#     if( sea_dataset_full.loc[i, phenotype] > thresholds[ key ] ):
#         sea_dataset_full.loc[i, "Case"] = True

# # Print summary of Case/Controls for each subgroup
# print( sea_dataset_full.groupby(["race","sex","Case"])["Case"].count() )
# sea_dataset_full

### --------------------------------------------------------------------------------

# Assign case/control classes using CR > 0 definition

# Phenotype of interest
phenotype = "cr"

# Add a new column called Case with all values equals to False
sea_dataset_full = sea_dataset_full.assign( Case=False )

# Identify subjects above threshold to assign them to group "Case"
for i in sea_dataset_full.index:
    
    if( sea_dataset_full.loc[i, phenotype] > 0 ):
        sea_dataset_full.loc[i, "Case"] = True

# Print summary of Case/Controls for each subgroup
print( sea_dataset_full.groupby(["race","sex","Case"])["Case"].count() )
sea_dataset_full

**Step 6)** **Standardize** raw scores and calculate **Odds Ratios** between case/controls

In [ ]:
# Standardize raw scores and calculate OR

li = []
standardize_scores = []
logit_models_dict = {}
models_summaries = pd.DataFrame()

# Get list of panels, thresholds and GRSs used from info dictonary
reference_panels = set( [ key.split('_')[0] for key in scores_info_dict.keys() ] )
thresholds = set( [ key.split('_')[1] for key in scores_info_dict.keys() ] )
grss = set( [ key.split('_')[2] for key in scores_info_dict.keys() ] )

# Normalize all scores with mean = 0 and SD = 1
for race in ["white", "black"]:
    
    # Subset cohort between whites and blacks to run logistic regression individually
    sea_dataset_subset = sea_dataset_full[ sea_dataset_full["race"] == race ]

    # Create a dependent variable named "Case_recoded" based on column "Case", conding it as 0 and 1 to fit glm
    sea_dataset_subset = sea_dataset_subset.assign( Case_recoded=sea_dataset_subset["Case"].replace(True, 1).replace(False, 0) )
    
    for reference_panel_id in reference_panels:
        
        for threshold in thresholds:

            for grs in grss:

                raw_score_name = "_".join( (reference_panel_id, threshold, grs) )
                standardize_score_name = "_".join( (race, reference_panel_id, threshold, grs) )
                standardize_scores.append(standardize_score_name)
        
                # Standardize raw scores using method scale
                sea_dataset_subset[ standardize_score_name ] = preprocessing.scale( sea_dataset_subset[ raw_score_name ] )
        
                # Fit a logistic model using standardize scores and save it on dictionary 
                logit_models_dict[ standardize_score_name ] = sm.formula.glm(
                    "Case_recoded ~ " + standardize_score_name + " + age + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10",
                    # "Case_recoded ~ " + standardize_score_name + " + age + sex + bmi + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10",
                    family=sm.families.Binomial(),
                    data=sea_dataset_subset).fit()

                # Create dictonary with scores performances
                model_summary_series = {
                        "model_ref": raw_score_name,
                        "race": race,
                        "reference_panel": reference_panel_id,
                        "threshold": threshold,
                        "GRS": grs,
                        "odds_ratio": np.exp( logit_models_dict[ standardize_score_name ].params )[ standardize_score_name ],
                        "score_pvalue": logit_models_dict[ standardize_score_name ].pvalues[ standardize_score_name ],
                        "conf_interval_lower": np.exp( logit_models_dict[ standardize_score_name ].conf_int()[0][ standardize_score_name ] ),
                        "conf_interval_upper": np.exp( logit_models_dict[ standardize_score_name ].conf_int()[1][ standardize_score_name ] )
                    }
                # Add additional info from info files (coverage, etc)
                model_summary_series.update( scores_info_dict[raw_score_name][race + "s"] )
                
                # Add metadata about the GRSs to the output
                model_summary_series.update( pgs_metadata[grs] )
                
                # Transform dictionary on a pandas Series and name it as <standardize_score_name>
                model_summary_series = pd.Series(
                    data=model_summary_series,
                    name=standardize_score_name
                )
                                
                li.append(model_summary_series)
                     
# Concatenate results for all models tested and transpose object to make visualization easier 
models_summaries = pd.concat( li, axis=1 ).transpose().sort_values("odds_ratio", ascending=False)

# Move trait_mapped column to first position to facilitate analysis of excel results
columns_order = models_summaries.columns.drop("trait_mapped").tolist()
columns_order.insert(0,"trait_mapped")
models_summaries = models_summaries.reindex(columns=columns_order, copy=False)
models_summaries

**Step 7)** Export dataframe with summaries as a **xls** file

In [ ]:
# Export dataframe with summaries on a xls file on the exports folder

writer = pd.ExcelWriter(
    path = "/labs/tassimes/rodrigoguarischi/projects/sea/exports/SEA_models_summary.xls",
    engine = 'xlsxwriter'
    )

models_summaries.to_excel(writer, sheet_name='models_summary')
# models_summaries_interaction.to_excel(writer, sheet_name='models_summary')

# Get access to the workbook and sheet
workbook = writer.book
worksheet = writer.sheets['models_summary']

# Automatically add filters to columns
worksheet.autofilter( "A1:AH{0}".format(len( models_summaries.index ) + 1) )

# Add a percent format with 2 decimal points
percent_fmt = workbook.add_format({'num_format': '0.00%', 'align': 'left'})

# Add a left-alignment format
left_alignment_fmt = workbook.add_format({'align': 'left'})

# Format the columns by width and include number formats
default_column_width = 11
worksheet.set_column('A:A', 35, left_alignment_fmt)                         # Model full name
worksheet.set_column('B:B', 45, left_alignment_fmt)                         # Trait mapped
worksheet.set_column('C:C', 30, left_alignment_fmt)                         # Model ref
worksheet.set_column('D:O', default_column_width, left_alignment_fmt)     
worksheet.set_column('P:P', default_column_width, percent_fmt)              # Coverage
worksheet.set_column('Q:AF', default_column_width, left_alignment_fmt)
worksheet.set_column('AG:AG', 70, left_alignment_fmt)                       # Citation
worksheet.set_column('AH:AH', 40, left_alignment_fmt)                       # PGS catalog link
worksheet.set_column('AI:AI', default_column_width, left_alignment_fmt)     # License

# Transform cells on column AH on hyperlinks
for model_index in range(0, len(models_summaries)):

    # Skip one line for the header +1 because excel location is 1-based
    row_number = model_index + 2
    model_index_name = models_summaries.index[model_index]
    
    # Get text to transform it to URL
    hyperlink = models_summaries.loc[model_index_name,"pgs_catalog_hyperlink"]
    
    # write text as hyperlink
    worksheet.write_url( "AH" + str(row_number), hyperlink)

## Add colors to cells of coverage labels to easy visualization

# Create a red, yellow and green fills with dark text
red_format = workbook.add_format({'bg_color':   '#FFC7CE', 'font_color': '#9C0006'})
yellow_format = workbook.add_format({'bg_color':   '#FFEB9C', 'font_color': '#9C6500'})
green_format = workbook.add_format({'bg_color':   '#C6EFCE', 'font_color': '#006100'})

# Get location of coverage label cells
cells_location = 'W1:W' + str( len(models_summaries) + 1)

worksheet.conditional_format( cells_location, {'type': 'cell', 'criteria': 'equal to', 'value': '"high"', 'format': green_format} )
worksheet.conditional_format( cells_location, {'type': 'cell', 'criteria': 'equal to', 'value': '"medium"', 'format': yellow_format} )
worksheet.conditional_format( cells_location, {'type': 'cell', 'criteria': 'equal to', 'value': '"low"', 'format': red_format} )
worksheet.conditional_format( cells_location, {'type': 'cell', 'criteria': 'equal to', 'value': '"zero"', 'format': red_format} )

# Save modifications
writer.save()

In [ ]:
### Plot selected scores segregrating cohort in whites and blacks 

# Set image resolution and background
from shutil import which

plt.rcParams['figure.dpi']= 150
sns.set_style('white')

# Function to return a subset of models that will be used on the forest plot
def get_models_subset(race, reference_panel_id, grs_subset, threshold ):
    
    # Get only subset of lines that meet the criteria
    models_subset = models_summaries[ (models_summaries["race"] == race) & (models_summaries["reference_panel"] == reference_panel_id) & (models_summaries["GRS"].isin( grs_subset) ) & (models_summaries["threshold"] == threshold )]
    models_subset = models_subset.sort_values("odds_ratio", ascending=False)
    
    return( models_subset )

# Define acronym and classes of selected scores
grs_subset_and_info = {
    "PGS000018": ["CAD      metaGRS    1,745,179", "CAD"], 
    "PGS000889": [" LDL          P+T           9,009    ", "Dyslipidemia"], 
    "PGS002133": ["Fat %     LDpred2        995,419", "Adiposity"],
    "PGS001351": ["  Ins         PRS-CS     1,025,098", "Dysglycemia"], 
    "PGS002161": ["BMI      LDpred2        990,022 ", "Adiposity"], 
    "PGS000667": ["Lp(a)          GWS            43      ", "Dyslipidemia"], 
    "PGS002197": [" Trig       LDpred2       731,035 ", "Dyslipidemia"], 
    "PGS002026": [" T2D      LDpred2       830,783 ", "Dysglycemia"], 
    "PGS002009": [" SBP        P. reg.          68,449  ", "Hypertension"]
    # "wGRS49": ["CAD  ", "CAD"], 
    # "TEMprsCatherineWhites": ["CAD  ", "CAD"], 
    # "PGS002150": [" LDL  ", "Dyslipidemia"], 
    # "PGS001933": [" LDL  ", "Dyslipidemia"]
    # "PGS002114": ["DBP", "Hypertension"]
    }
class_to_color = {
    "CAD": sns.color_palette( n_colors=40, palette="CMRmap" )[19],
    "Adiposity": sns.color_palette( n_colors=40, palette="CMRmap" )[32],
    "Dysglycemia": sns.color_palette( n_colors=40, palette="CMRmap" )[13],
    "Dyslipidemia": sns.color_palette( n_colors=40, palette="Dark2" )[4],
    "Hypertension": sns.color_palette( n_colors=10, palette="Blues" )[6]
    }

for race in ['white', "black"]:

    # Subset based on criterias and save to a new dataframe
    models_subset = get_models_subset( race, "topmed", grs_subset_and_info.keys(), "r0")
    
    # Assign classes and colors to each score
    models_subset = models_subset.assign( acronym=[ grs_subset_and_info[ elem ][0] for elem in models_subset["GRS"].to_list() ] )
    models_subset = models_subset.assign( grs_class=[ grs_subset_and_info[ elem ][1] for elem in models_subset["GRS"].to_list() ] )
    models_subset = models_subset.assign( color=[ class_to_color[ elem ] for elem in models_subset["grs_class"].to_list() ] )
    
    # Test p-values to see if they survive to multiple testing correction (FDR)
    fdr_result = fdrcorrection( models_subset["score_pvalue"].to_list() )
    models_subset = models_subset.assign( adjusted_p=fdr_result[1] )
    models_subset = models_subset.assign( fdr_significant=fdr_result[0] )

    # # Print p-values/q-values
    # print("Race: " + race )
    # print("p-values:\n" + "\n".join(map(str, models_subset["score_pvalue"].to_list()) ) )
    # print("\nq-values:\n" + "\n".join(map(str, fdr_result[1] ) ) + "\n")
    
    # Create plot
    p = EffectMeasurePlot(
        label=[ "{0}     {1}       {2:.3f}{3}".format(row["GRS"], row["acronym"], row["score_pvalue"], "*" if( row["fdr_significant"] ) else "  " ) for index, row in models_subset.iterrows() ], 
        # label=[ "{0}    {1:.2e}{2}".format(row["pgs_name"], row["score_pvalue"], "*" if( row["fdr_significant"] ) else "  " ) for index, row in models_subset.iterrows() ], 
        effect_measure=[ "{0:.2f}".format(row["odds_ratio"]) for index, row in models_subset.iterrows() ],
        lcl=[ "{0:.2f}".format(row["conf_interval_lower"]) for index, row in models_subset.iterrows() ],
        ucl=[ "{0:.2f}".format(row["conf_interval_upper"]) for index, row in models_subset.iterrows() ]
        )
    # Adjust axis, labels, titles and frame
    p.labels(effectmeasure='OR', scale='log')
    p.colors(pointshape="D", pointcolor=models_subset["color"], errorbarcolor=models_subset["color"] )
    ax=p.plot(figsize=(7,3), t_adjuster=0.05, min_value=0.5, max_value=2)    
    plt.suptitle(" PGS ID   Trait  Method  N variants  p-value   ", x=-0.14, y=0.90, fontweight="bold")
    number_of_cases = len( sea_dataset_full[ ( sea_dataset_full["race"] == race ) & ( sea_dataset_full["Case"] == True ) ] )
    number_of_controls = len( sea_dataset_full[ ( sea_dataset_full["race"] == race ) & ( sea_dataset_full["Case"] == False ) ] )
    plt.title("{0} ({1} cases, {2} controls)         Odds Ratio per SD increase".format(race.capitalize(), number_of_cases, number_of_controls), loc="right",x=1.2, y=1.02)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(False)
    ax.tick_params( axis='x', which="minor", bottom=False )
        
# Add legend
trait_handles = []
for grs_class in sorted( set( models_subset[ "grs_class" ] )  ):
    trait_handles.append( mlines.Line2D([], [], label=grs_class, color=class_to_color[grs_class], marker='_', markersize=15) )
ax.legend(handles=trait_handles, loc=(-0.5, -0.3), shadow=False, borderpad=0.6, ncol=5)


In [ ]:
### Histogram plots with raw scores

fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(12,5))

for score in ["topmed_r0_PGS000018", "topmed_r0_PGS000889"]:

    sns.histplot(
        x = score,
        hue = "race",
        kde=True,
        data = sea_dataset_full,
        ax = ax1 if score == "topmed_r0_PGS000018" else ax2
        )

In [ ]:
### Plot score quantiles using % of cohort where Case = True

def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'Q_%s' % n
    return percentile_

def cases_percentage(x):
    frac = sum(x)/len(x)
    return frac

def show_values(axs, orient="v", space=.01):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                value = '{:.1f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.1f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)

# Set image resolution and background
plt.rcParams['figure.dpi']= 150
sns.set_style('white')

# Create a 2 panel plot
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False, figsize=(12,5))

# Define for which race subgroup to make barplots
race = "white"

## Plot score quantiles using % of cohort where Case = True

# Subset cohort to whites only
# sea_dataset_subset = sea_dataset_full
sea_dataset_subset = sea_dataset_full[ sea_dataset_full["race"] == race ]

# Split data on non equally-sized quantiles (20-60-20)
quantile_labels = ["Q1", "Q2-Q4", "Q5"]
sea_dataset_subset = sea_dataset_subset.assign( CAD=pd.qcut(sea_dataset_subset["topmed_r0_PGS000018"], [0,0.2,0.8,1], labels=quantile_labels) )
sea_dataset_subset = sea_dataset_subset.assign( LDL=pd.qcut(sea_dataset_subset["topmed_r0_PGS000889"], [0,0.2,0.8,1], labels=quantile_labels) )

cases_quantiles = pd.concat( [
        sea_dataset_subset.groupby(["CAD"])["Case"].agg( [ "count", "sum", cases_percentage ] ).assign( score="CAD", quantile=quantile_labels ),
        sea_dataset_subset.groupby(["LDL"])["Case"].agg( [ "count", "sum", cases_percentage ] ).assign( score="LDL", quantile=quantile_labels )
    ] )

# Transform value in percentage
cases_quantiles["cases_percentage"] = cases_quantiles["cases_percentage"] * 100

# Print some basic statistics
print( cases_quantiles )
    
# Plot 
p = sns.barplot(
    x = "quantile",
    y = "cases_percentage",
    hue = "score",
    ci = None,
    palette = {
        "CAD": sns.color_palette( n_colors=40, palette="CMRmap" )[19],
        "LDL": sns.color_palette( n_colors=40, palette="Dark2" )[4]
    },
    data = cases_quantiles,
    ax = ax1
    )

# Change axis labels
p.set( xlabel = "Score quintiles", ylabel = "Prevalence of raised coronary lesions (%)" )

# Display value on top of barplots
show_values(p)
 
## Plot score quantiles using median % of surface with RC (only CR+ cases)

# Subset cohort to whites only
# sea_dataset_subset = sea_dataset_full
sea_dataset_subset = sea_dataset_full[ ( sea_dataset_full["race"] == race ) & (sea_dataset_full["Case"] == True) ]

# Split data on non equally-sized quantiles (20-60-20)
quantile_labels = ["Q1", "Q2-Q4", "Q5"]
sea_dataset_subset = sea_dataset_subset.assign( CAD=pd.qcut(sea_dataset_subset["topmed_r0_PGS000018"], [0,0.2,0.8,1], labels=quantile_labels) )
sea_dataset_subset = sea_dataset_subset.assign( LDL=pd.qcut(sea_dataset_subset["topmed_r0_PGS000889"], [0,0.2,0.8,1], labels=quantile_labels) )

# Print some basic statistics
y_variable = "cr"
print( sea_dataset_subset.groupby( [ "race", "CAD" ] )[y_variable].agg( [ "count", "min", percentile(25), "median", percentile(75), "mean", "max",  ] ).round(2) )
print( sea_dataset_subset.groupby( [ "race", "LDL" ] )[y_variable].agg( [ "count", "min", percentile(25), "median", percentile(75), "mean", "max",  ] ).round(2) )

# Generate barplot of score quantiles using median % of surface with RC
p = sns.barplot(
    x = 'value',
    y = y_variable,
    hue = 'variable',
    ci = None,
    estimator=np.median,
    order = quantile_labels,
    palette = {
        "CAD": sns.color_palette( n_colors=40, palette="CMRmap" )[19],
        "LDL": sns.color_palette( n_colors=40, palette="Dark2" )[4]
        },
    data = pd.melt( sea_dataset_subset, id_vars=[y_variable], value_vars=['CAD', 'LDL'], ignore_index=False ),
    ax = ax2
    )

# Change axis labels and ranges
# p.set( xlabel="Score quintiles", ylabel="Mean surface area involvement (%)", ylim=(0,18) )
p.set( xlabel="Score quintiles", ylabel="Median surface area involvement (%)", ylim=(0,8) )

# Display value on top of barplots
show_values(p)

# Add a single legend for both panels
trait_handles = []
trait_handles.append( mlines.Line2D([], [], label="CAD (PGS000018)", color=sns.color_palette( n_colors=40, palette="CMRmap" )[19], lw=8) )
trait_handles.append( mlines.Line2D([], [], label="LDL (PGS000889)", color=sns.color_palette( n_colors=40, palette="Dark2" )[4], lw=8) )
ax1.legend_.remove()
ax2.legend( title="Polygenic score", handles=trait_handles, loc=(-0.53, -0.25), shadow=False, borderpad=0.6, ncol=3 )

Extra: Test interaction term **score*race**

In [ ]:
### Test interaction term score*race

# Standardize raw scores and calculate OR

li = []
standardize_scores = []
logit_models_interaction_dict = {}
models_summaries_interaction = pd.DataFrame()

# Get list of panels, thresholds and GRSs used from info dictonary
reference_panels = set( [ key.split('_')[0] for key in scores_info_dict.keys() ] )
thresholds = set( [ key.split('_')[1] for key in scores_info_dict.keys() ] )
grss = set( [ key.split('_')[2] for key in scores_info_dict.keys() ] )

# For interaction term testing, we are testing the full cohort (whites and blacks) together. 
sea_dataset_test_interaction = sea_dataset_full

# Create a dependent variable named "Case_recoded" based on column "Case", conding it as 0 and 1 to fit glm
sea_dataset_test_interaction = sea_dataset_test_interaction.assign( Case_recoded=sea_dataset_test_interaction["Case"].replace(True, 1).replace(False, 0) )

for reference_panel_id in reference_panels:
    
    for threshold in thresholds:

        for grs in grss:

            raw_score_name = "_".join( (reference_panel_id, threshold, grs) )
            standardize_score_name = "_".join( (reference_panel_id, threshold, grs) )
            standardize_scores.append(standardize_score_name)
        
            # Standardize raw scores using method scale (mean = 0 and SD = 1)
            sea_dataset_test_interaction[ standardize_score_name ] = preprocessing.scale( sea_dataset_test_interaction[ raw_score_name ] )
        
            # Fit a logistic model using standardize scores testing for interaction between score and race 
            logit_models_interaction_dict[ standardize_score_name ] = sm.formula.glm(
                # "Case_recoded ~ " + standardize_score_name + "*race + age + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10",
                "Case_recoded ~ " + standardize_score_name + " + race + age + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10",
                family=sm.families.Binomial(),
                data=sea_dataset_test_interaction).fit()

            # Create dictonary with scores performances
            model_summary_series = {
                    "model_ref": raw_score_name,
                    "reference_panel": reference_panel_id,
                    "threshold": threshold,
                    "GRS": grs,
                    "odds_ratio": np.exp( logit_models_interaction_dict[ standardize_score_name ].params )[ standardize_score_name ],
                    "score_pvalue": logit_models_interaction_dict[ standardize_score_name ].pvalues[ standardize_score_name ],
                    "conf_interval_lower": np.exp( logit_models_interaction_dict[ standardize_score_name ].conf_int()[0][ standardize_score_name ] ),
                    "conf_interval_upper": np.exp( logit_models_interaction_dict[ standardize_score_name ].conf_int()[1][ standardize_score_name ] )
                }
                
            # Add metadata about the GRSs to the output
            model_summary_series.update( pgs_metadata[grs] )
                
            # Transform dictionary on a pandas Series and name it as <standardize_score_name>
            model_summary_series = pd.Series(
                data=model_summary_series,
                name=standardize_score_name
            )
                                
            li.append(model_summary_series)
                     
# Concatenate results for all models tested and transpose object to make visualization easier 
models_summaries_interaction = pd.concat( li, axis=1 ).transpose().sort_values("odds_ratio", ascending=False)

# Move trait_mapped column to first position to facilitate analysis of excel results
columns_order = models_summaries_interaction.columns.drop("trait_mapped").tolist()
columns_order.insert(0,"trait_mapped")
models_summaries_interaction = models_summaries_interaction.reindex(columns=columns_order, copy=False)
# models_summaries_interaction


# List of GRS to print results
grs_subset = [ "PGS000018", "PGS000889", "PGS002133", "PGS001351", "PGS002161", "PGS000667", "PGS002197", "PGS002009", "PGS002026" ]

for grs_id in grs_subset:
    full_grs_id = "topmed_r0_" + grs_id
    print( ">>> ",  full_grs_id , "\n", logit_models_interaction_dict[ full_grs_id ].summary(), "\n")

# # SIRE logist 
# grs_id = "white_topmed_r0_PGS000018"; print( ">>> ", grs_id, "\n", logit_models_dict[ grs_id ].summary(), "\n")
# grs_id = "black_topmed_r0_PGS000018"; print( ">>> ", grs_id, "\n", logit_models_dict[ grs_id ].summary(), "\n")

# # Interaction score*sire
# grs_id = "topmed_r0_PGS000018"; print( ">>> ", grs_id, "\n", logit_models_interaction_dict[ grs_id ].summary(), "\n")
# grs_id = "topmed_r0_PGS000889"; print( ">>> ", grs_id, "\n", logit_models_interaction_dict[ grs_id ].summary(), "\n")
# grs_id = "topmed_r0_PGS002133"; print( ">>> ", grs_id, "\n", logit_models_interaction_dict[ grs_id ].summary(), "\n")
# grs_id = "topmed_r0_PGS002161"; print( ">>> ", grs_id, "\n", logit_models_interaction_dict[ grs_id ].summary(), "\n")

# print( logit_models_interaction_dict[ "" ].summary() )
# print( logit_models_interaction_dict[ "topmed_r0_PGS000018" ].summary() )



In [ ]:
# Set image resolution and background
plt.rcParams['figure.dpi']= 150
sns.set_style('white')

# for dataset in sea_phenotypes, sea_phenotypes[sea_phenotypes["cr"] > 0]: 
# for dataset in sea_phenotypes, sea_phenotypes:

plot_dataset = sea_phenotypes
# plot_dataset = sea_phenotypes[sea_phenotypes["cr"] > 0]

n_white = len( plot_dataset[plot_dataset["race"] == "white"] )
n_black = len( plot_dataset[plot_dataset["race"] == "black"] )

sns.histplot(
       x = "cr",
       data = plot_dataset,
       hue="race",
       multiple="dodge",
       shrink=0.8,
       # log_scale=(False,True),
       # bins=[0,1,5,20,50,100],
       binrange=(0,100),
       binwidth=10
       ).set( 
             title='Raised coronary lesions (white={0}, black={1})'.format( n_white, n_black ), 
             xlabel="Surface area involvement (%)"
             )

In [ ]:
### Plot selected scores without segregrating cohort in whites and blacks 

# Set image resolution and background
plt.rcParams['figure.dpi']= 150
sns.set_style('white')

# Function to return a subset of models that will be used on the forest plot
def get_models_subset(reference_panel_id, grs_subset, threshold ):
    
    # Get only subset of lines that meet the criteria
    models_subset = models_summaries_interaction[ (models_summaries_interaction["reference_panel"] == reference_panel_id) & (models_summaries_interaction["GRS"].isin( grs_subset) ) & (models_summaries_interaction["threshold"] == threshold )]
    models_subset = models_subset.sort_values("odds_ratio", ascending=False)
    
    return( models_subset )

# Define acronym and classes of selected scores
grs_subset_and_info = {
    "PGS000018": ["CAD  ", "CAD"], 
    "PGS000889": [" LDL  ", "Dyslipidemia"], 
    "PGS002133": ["Fat %", "Adiposity"],
    "PGS001351": ["  Ins  ", "Dysglycemia"], 
    # "PGS002114": ["DBP", "Hypertension"], 
    "PGS002161": [" BMI  ", "Adiposity"], 
    "PGS000667": ["Lp(a) ", "Dyslipidemia"], 
    "PGS002197": [" Trig  ", "Dyslipidemia"], 
    "PGS002026": [" T2D  ", "Dysglycemia"], 
    "PGS002009": [" SBP  ", "Hypertension"]
    }
class_to_color = {
    "CAD": sns.color_palette( n_colors=40, palette="CMRmap" )[19],
    "Adiposity": sns.color_palette( n_colors=40, palette="CMRmap" )[32],
    "Dysglycemia": sns.color_palette( n_colors=40, palette="CMRmap" )[13],
    "Dyslipidemia": sns.color_palette( n_colors=40, palette="Dark2" )[4],
    "Hypertension": sns.color_palette( n_colors=10, palette="Blues" )[6]
    }

# Subset based on criterias and save to a new dataframe
models_subset = get_models_subset( "topmed", grs_subset_and_info.keys(), "r0")
    
# Assign classes and colors to each score
models_subset = models_subset.assign( acronym=[ grs_subset_and_info[ elem ][0] for elem in models_subset["GRS"].to_list() ] )
models_subset = models_subset.assign( grs_class=[ grs_subset_and_info[ elem ][1] for elem in models_subset["GRS"].to_list() ] )
models_subset = models_subset.assign( color=[ class_to_color[ elem ] for elem in models_subset["grs_class"].to_list() ] )

# Test p-values to see if they survive to multiple testing correction (FDR)
fdr_result = fdrcorrection( models_subset["score_pvalue"].to_list() )
models_subset = models_subset.assign( adjusted_p=fdr_result[1] )
models_subset = models_subset.assign( fdr_significant=fdr_result[0] )

# Create plot
p = EffectMeasurePlot(
    label=[ "{0}     {1}       {2:.3f}{3}".format(row["GRS"], row["acronym"], row["score_pvalue"], "*" if( row["fdr_significant"] ) else "  " ) for index, row in models_subset.iterrows() ], 
    effect_measure=[ "{0:.2f}".format(row["odds_ratio"]) for index, row in models_subset.iterrows() ],
    lcl=[ "{0:.2f}".format(row["conf_interval_lower"]) for index, row in models_subset.iterrows() ],
    ucl=[ "{0:.2f}".format(row["conf_interval_upper"]) for index, row in models_subset.iterrows() ]
    )
# Adjust axis, labels, titles and frame
p.labels(effectmeasure='OR', scale='log')
p.colors(pointshape="D", pointcolor=models_subset["color"], errorbarcolor=models_subset["color"] )
ax=p.plot(figsize=(7,3), t_adjuster=0.05, min_value=0.5, max_value=2)    
plt.suptitle("     PGS ID    Trait    p-value", x=-0.05, y=0.90, fontweight="bold")
plt.title("Whites+Blacks                     Odds Ratio per SD increase", loc="right",x=1.2, y=1.02)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)
        
# Add legend
trait_handles = []
for grs_class in sorted( set( models_subset[ "grs_class" ] )  ):
    trait_handles.append( mlines.Line2D([], [], label=grs_class, color=class_to_color[grs_class], marker='_', markersize=15) )
ax.legend(handles=trait_handles, loc=(-0.5, -0.3), shadow=False, borderpad=0.6, ncol=5)

In [ ]:
# import zepid
# from zepid.graphics import EffectMeasurePlot

# # Function to return a subset of models that will be used on the forest plot
# def get_models_subset(race, reference_panel_id, grs_subset, threshold ):
    
#     # Get only subset of lines that meet the criteria
#     models_subset = models_summaries[ (models_summaries["race"] == race) & (models_summaries["reference_panel"] == reference_panel_id) & (models_summaries["GRS"].isin( grs_subset) ) & (models_summaries["threshold"] == threshold )]
#     models_subset = models_subset.sort_values("odds_ratio", ascending=False)
    
#     return( models_subset )

# # List of GRS to include
# grs_subset = [ "PGS002026", "PGS000889", "PGS002161", "PGS002114", "PGS001351", "PGS000018", "PGS002197", "PGS000667", "PGS002133" ]

# for race in ['white', "black"]:

#     for panel in ["topmed", "hrc"]:

#         # Subset based on criterias and save to a new dataframe
#         models_subset = get_models_subset( race, panel, grs_subset, "r0")

#         # Create plot
#         p = EffectMeasurePlot(
#             # label=[ "{0}  ({1:.2e})".format(row["GRS"], row["score_pvalue"]) for index, row in models_subset.iterrows() ], 
#             # label=[ "{0}    {1:.2e}".format(row["pgs_name"], row["score_pvalue"]) for index, row in models_subset.iterrows() ], 
#             label=[ "{0}|{1}    {2:.2e}".format(row["pgs_name"], row["GRS"], row["score_pvalue"]) for index, row in models_subset.iterrows() ], 
#             effect_measure=[ "{0:.2f}".format(row["odds_ratio"]) for index, row in models_subset.iterrows() ],
#             lcl=[ "{0:.2f}".format(row["conf_interval_lower"]) for index, row in models_subset.iterrows() ],
#             ucl=[ "{0:.2f}".format(row["conf_interval_upper"]) for index, row in models_subset.iterrows() ]
#             )

#         # Adjust axis, labels, titles and frame
#         p.labels(effectmeasure='OR')
#         p.colors(pointshape="D")
#         ax=p.plot(figsize=(7,3), t_adjuster=0.05, min_value=0.75, max_value=2 )
#         # plt.suptitle("Polygenic Score - {0}s   p-value".format(race), x=-0.09, y=0.98)
#         plt.suptitle("Polygenic Score - {0}s ({1})   p-value".format(race, panel), x=-0.15, y=0.98)
#         plt.title("Odds Ratio per SD increase",loc="center",x=0.6, y=1.10)
#         ax.spines['top'].set_visible(False)
#         ax.spines['right'].set_visible(False)
#         ax.spines['bottom'].set_visible(True)
#         ax.spines['left'].set_visible(False)


In [ ]:
# ## OLD: Plot all scores and colored by traits ###
# #################################################

# import zepid
# from zepid.graphics import EffectMeasurePlot
# import matplotlib.lines as mlines

# # Function to return a subset of models that will be used on the forest plot
# def get_models_subset(race, reference_panel_id, grs_subset, threshold ):
    
#     # Get only subset of lines that meet the criteria
#     models_subset = models_summaries[ (models_summaries["race"] == race) & (models_summaries["reference_panel"] == reference_panel_id) & (models_summaries["GRS"].isin( grs_subset) ) & (models_summaries["threshold"] == threshold )]
#     models_subset = models_subset.sort_values("odds_ratio", ascending=False)
    
#     return( models_subset )

# # List of GRS to include
# # grs_subset = [ "PGS002026", "PGS000889", "PGS002161", "PGS002114", "PGS001351", "PGS000018", "PGS002197", "PGS000667", "PGS002133" ]
# grs_subset = set( models_summaries["GRS"].to_list() )

# for race in ['white', "black"]:

#     # Subset based on criterias and save to a new dataframe
#     models_subset = get_models_subset( race, "topmed", grs_subset, "r0")
    
#     traits_set = set( models_subset["trait_mapped"] )
#     trait_2_colors = dict( zip( traits_set, sns.color_palette(n_colors=len(traits_set) ) ) )

#     # Create plot
#     p = EffectMeasurePlot(
#         # label=[ "{0}  {1:.2e}".format(row["GRS"], row["score_pvalue"]) for index, row in models_subset.iterrows() ], 
#         label=[ "{0}    {1:.2e}".format(row["pgs_name"], row["score_pvalue"]) for index, row in models_subset.iterrows() ], 
#         effect_measure=[ "{0:.2f}".format(row["odds_ratio"]) for index, row in models_subset.iterrows() ],
#         lcl=[ "{0:.2f}".format(row["conf_interval_lower"]) for index, row in models_subset.iterrows() ],
#         ucl=[ "{0:.2f}".format(row["conf_interval_upper"]) for index, row in models_subset.iterrows() ]
#         )
#     # Adjust axis, labels, titles and frame
#     p.labels(effectmeasure='OR')
#     p.colors(pointshape="D", pointcolor=[ trait_2_colors[elem] for elem in models_subset["trait_mapped"].to_list() ], errorbarcolor=[ trait_2_colors[elem] for elem in models_subset["trait_mapped"].to_list() ])
#     ax=p.plot(figsize=(7,15), t_adjuster=0.01, min_value=0.75, max_value=2 )
    
#     plt.suptitle("Polygenic Score - {0}s   p-value".format(race), x=-0.09, y=0.90)
#     plt.title("Odds Ratio per SD increase",loc="center",x=0.6, y=1.02)
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.spines['bottom'].set_visible(True)
#     ax.spines['left'].set_visible(False)
    
#     # Add legend
#     trait_handles = []
#     for trait in sorted( trait_2_colors.keys() ):
#         trait_handles.append( mlines.Line2D([], [], label=trait, color=trait_2_colors[trait], marker='_', markersize=15) )
#     ax.legend(handles=trait_handles, loc=(1.65, 0.45), shadow=True, borderpad=0.6)    


In [ ]:
# ## OLD: Plot all scores and colored by imputation reference panel ###
# #####################################################################

# import zepid
# from zepid.graphics import EffectMeasurePlot
# import matplotlib.lines as mlines

# # Function to return a subset of models that will be used on the forest plot
# def get_models_subset(race, grs_subset, threshold ):
    
#     # Get only subset of lines that meet the criteria
#     models_subset = models_summaries[ (models_summaries["race"] == race) & (models_summaries["GRS"].isin( grs_subset) ) & (models_summaries["threshold"] == threshold )]
#     models_subset = models_subset.sort_values("odds_ratio", ascending=False)
    
#     return( models_subset )

# # List of GRS to include
# # grs_subset = [ "PGS002026", "PGS000889", "PGS002161", "PGS002114", "PGS001351", "PGS000018", "PGS002197", "PGS000667", "PGS002133" ]
# grs_subset = set( models_summaries["GRS"].to_list() )

# panel_2_colors = {
#     'topmed': sns.color_palette(n_colors=2)[0],
#     'hrc': sns.color_palette(n_colors=2)[1]
# }

# for race in ['white', "black"]:

#     # Subset based on criterias and save to a new dataframe
#     models_subset = get_models_subset( race, grs_subset, "r0")
#     # Create plot
#     p = EffectMeasurePlot(
#         # label=[ "{0}  ({1:.2e})".format(row["GRS"], row["score_pvalue"]) for index, row in models_subset.iterrows() ], 
#         label=[ "{0}    {1:.2e}".format(row["pgs_name"], row["score_pvalue"]) for index, row in models_subset.iterrows() ], 
#         effect_measure=[ "{0:.2f}".format(row["odds_ratio"]) for index, row in models_subset.iterrows() ],
#         lcl=[ "{0:.2f}".format(row["conf_interval_lower"]) for index, row in models_subset.iterrows() ],
#         ucl=[ "{0:.2f}".format(row["conf_interval_upper"]) for index, row in models_subset.iterrows() ]
#         )
#     # Adjust axis, labels, titles and frame
#     p.labels(effectmeasure='OR')
#     p.colors(pointshape="D", pointcolor=[ panel_2_colors[elem] for elem in models_subset["reference_panel"].to_list() ], errorbarcolor=[ panel_2_colors[elem] for elem in models_subset["reference_panel"].to_list() ])
#     ax=p.plot(figsize=(7,15), t_adjuster=0.01, min_value=0.75, max_value=2 )
    
#     plt.suptitle("Polygenic Score - {0}s   p-value".format(race), x=-0.09, y=0.90)
#     plt.title("Odds Ratio per SD",loc="center",x=0.6, y=1.02)
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.spines['bottom'].set_visible(True)
#     ax.spines['left'].set_visible(False)
    
#     topmed_color = mlines.Line2D([], [], color=panel_2_colors['topmed'], marker='_', markersize=15, label='TOPMed')
#     hrc_color = mlines.Line2D([], [], color=panel_2_colors['hrc'], marker='_', markersize=15, label='HRC')
#     ax.legend(handles=[topmed_color, hrc_color], loc=(1.6,0.5), shadow=True, borderpad=0.6)


<h2>Extra analysis</h2>

Extra 1: Plot boxplots and histograms and boxplots of raw scores spliting by sex

In [ ]:
# Plot boxplots and histograms and boxplots of raw scores spliting by sex

sns.set_style('whitegrid')

score_list = ["wGRS49_r0", "wGRS49_r03", "wGRS49_r05", "wGRS49_r08"]
# score_list = ["PGS000349_r0", "PGS000349_r03", "PGS000349_r05", "PGS000349_r08"]
# score_list = ["PGS000018_r0", "PGS000018_r03", "PGS000018_r05", "PGS000018_r08"]

fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(5*len(score_list),12))
for i in range(0, len(score_list)):
    sns.boxplot(
        x = "Case",
        y = score_list[i],
        data = sea_merged_whites,
        palette = reversed(sns.color_palette(n_colors=2)),
        width=0.4,
        fliersize=2,
        ax=axs[0,i],
    )
    sns.histplot(
        x = score_list[i],
        data = sea_merged_whites[ sea_merged_whites["sex"] == "Male" ],
        hue="Case",
        hue_order=[True, False],
        kde=True,
        ax=axs[1,i]
        ).set(title='Males (n={0})'.format( sum( sea_merged_whites["sex"] == "Male" ) ) )
    sns.histplot(
        x = score_list[i],
        data = sea_merged_whites[ sea_merged_whites["sex"] == "Female" ],
        hue="Case",
        hue_order=[True, False],
        kde=True,
        ax=axs[2,i]
    ).set(title='Females (n={0})'.format( sum( sea_merged_whites["sex"] == "Female" ) ) )

plt.subplots_adjust(top=1.25)

# # Plot only boxplots

# fig, axs = plt.subplots(ncols=len(score_list), figsize=(5*len(score_list),4))
# for i in range(0, len(score_list)):
#     sns.boxplot(
#         x = "Case",
#         y = score_list[i],
#         data = sea_merged_whites,
#         hue="sex",
#         hue_order=['Male','Female'],
#         width=0.4,
#         fliersize=2,
#         ax=axs[i],
#     )

Extra 2: Compare OR from models with 5 PCs and 10 PCs

In [ ]:
# Need to run model calculation twice (5 PCs and 10 PCs) to be able to complete the cell on this analysis

# models_summaries_10pcs = models_summaries
# models_summaries_5pcs = models_summaries

import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi']= 150

sns.set_style('whitegrid')

pandas_5_10_pcs = pd.concat( [models_summaries_5pcs.add_suffix("_5PCs"), models_summaries_10pcs.add_suffix("_10PCs")], axis=1 )
pandas_5_10_pcs

# Create a column named race to use in plotting
pandas_5_10_pcs["race"] = pandas_5_10_pcs["race_5PCs"]

# Cast variables to float for plotting
pandas_5_10_pcs = pandas_5_10_pcs.astype( {
    "odds_ratio_5PCs": float,
    "odds_ratio_10PCs": float,
    "score_pvalue_5PCs": float
    } )

# Plot regression plot
sns.lmplot(
    x = "odds_ratio_5PCs",
    y = "odds_ratio_10PCs",
    data = pandas_5_10_pcs[ pandas_5_10_pcs["score_pvalue_10PCs"] <= 0.05 ],
    # data = pandas_5_10_pcs,
    hue="race", 
    scatter_kws={'s':50, 'alpha':0.5},
    line_kws={'lw':2.5, 'ls': '--'}
    )

# Set x and y limits and a diagonal line
axes_limits=(0.7, 1.6)
plt.plot(axes_limits, axes_limits, c='black', alpha=0.8, lw=1)

Extra 3: Calculate and plot beta differences by R thresholds for each model, panel, race

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import math

# Clone models_summaries adding a new column named beta_diff
beta_diffs = models_summaries.assign( beta_diff=np.nan )

# Function to return OR value for a given model
def get_beta_value(race, reference_panel_id, grs, threshold ):
    
    # Get OR value for a given model
    or_value = models_summaries[ (models_summaries["race"] == race) & (models_summaries["reference_panel"] == reference_panel_id) & (models_summaries["GRS"] == grs ) & (models_summaries["threshold"] == threshold )]["odds_ratio"]
    
    # Transform OR to Beta
    beta_value = math.log( or_value ) 
    
    return( beta_value )

# Loop over multiple models to get differences on betas
for race in models_summaries['race'].unique().tolist():
        
    for reference_panel_id in models_summaries['reference_panel'].unique().tolist():
        
        for grs in models_summaries['GRS'].unique().tolist():
            
            # Get baseline beta (r = 0)
            baseline_beta = get_beta_value( race, reference_panel_id, grs, "r0" )
            
            for threshold in models_summaries['threshold'].unique().tolist():
                
                r_beta = get_beta_value( race, reference_panel_id, grs, threshold )
                
                beta_diffs.loc[ "_".join( [race, reference_panel_id, threshold, grs] ) , "beta_diff"] = r_beta - baseline_beta
                

# Plot betas differences for each model | panel

plt.rcParams['figure.dpi']= 100

sns.set_style('whitegrid')

g = sns.FacetGrid(
    beta_diffs, 
    row="GRS",
    col="reference_panel",
    sharex=False,
    hue = "race",
    hue_order=['white','black']
    )

g.map_dataframe(
    sns.stripplot,
    x = "threshold",
    y = "beta_diff",
    order = ['r0', 'r03', 'r05', 'r08']
    )

g.set_titles( col_template="{col_name}", row_template="{row_name}" )

g.add_legend()